In [ ]:
# IR Components
# 1. Crawler + Indexer -> Crawler / Indexer
# 2. Doc Analyzer -> (Improved) BoW
# 3. Query -> 2번과 동일 -> DTM(TDM)
# 4. Ranking Model

## 1-1. (Focused) Crawler (탐색기법: BFS) - 네이버 뉴스

In [1]:
from requests import get
from requests.compat import urljoin
from os import listdir
from bs4 import BeautifulSoup

urls = ['https://news.naver.com']
visited = list() # 갔던 목록 필요하므로

while urls:
    seed = urls.pop(0) # Queue
    visited.append(seed)
    
    dom = BeautifulSoup(get(seed).text, 'html.parser')

In [2]:
dom.select('#right.ranking_contents')

[<div id="right.ranking_contents"></div>]

In [3]:
dom.select('div[id^=ranking_]')

[<div id="ranking_100" style="display:none">
 <h5 class="blind">정치</h5>
 <ul class="section_list_ranking">
 <li><span class="rank num1"><em>1</em></span> <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=081&amp;aid=0003114437&amp;date=20200810&amp;type=1&amp;rankingSeq=1&amp;rankingSectionId=100" title="[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”">[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”</a> </li>
 <li><span class="rank num2"><em>2</em></span> <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=215&amp;aid=0000892691&amp;date=20200810&amp;type=1&amp;rankingSeq=2&amp;rankingSectionId=100" title='[속보] 김진애 "다주택자 왜 적대시하냐, 노영민 과했다"'>[속보] 김진애 "다주택자 왜 적대시하냐, 노영민 과했다"</a> </li>
 <li><span class="rank num3"><em>3</em></span> <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=001&amp;aid=0011804920&amp;date=

In [4]:
len(dom.select('div[id^=ranking_]'))

6

In [5]:
dom.select('div[id^=ranking_]')[0].select('li > a')

[<a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=081&amp;aid=0003114437&amp;date=20200810&amp;type=1&amp;rankingSeq=1&amp;rankingSectionId=100" title="[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”">[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”</a>,
 <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=215&amp;aid=0000892691&amp;date=20200810&amp;type=1&amp;rankingSeq=2&amp;rankingSectionId=100" title='[속보] 김진애 "다주택자 왜 적대시하냐, 노영민 과했다"'>[속보] 김진애 "다주택자 왜 적대시하냐, 노영민 과했다"</a>,
 <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=001&amp;aid=0011804920&amp;date=20200810&amp;type=1&amp;rankingSeq=3&amp;rankingSectionId=100" title='문대통령 "집값 진정 양상…부동산감독기구 설치"(종합)'>문대통령 "집값 진정 양상…부동산감독기구 설치"(종합)</a>,
 <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oi

In [6]:
len(dom.select('div[id^=ranking_]')[0].select('li > a'))

10

In [7]:
dom.select('div[id^=ranking_]')[0].select('li > a')[0]

<a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=081&amp;aid=0003114437&amp;date=20200810&amp;type=1&amp;rankingSeq=1&amp;rankingSectionId=100" title="[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”">[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”</a>

In [8]:
for news in dom.select('div[id^=ranking_]'):
    print(news.attrs)

{'id': 'ranking_100', 'style': 'display:none'}
{'id': 'ranking_101', 'style': 'display:none'}
{'id': 'ranking_102', 'style': 'display:none'}
{'id': 'ranking_103', 'style': 'display:none'}
{'id': 'ranking_104', 'style': 'display:none'}
{'id': 'ranking_105', 'style': 'display:none'}


In [9]:
import re
from string import punctuation
pattern1 = re.compile('[{}]'.format(re.escape(punctuation)))

In [10]:
for news in dom.select('div[id^=ranking_]'):
    print(re.search('\d{3}', news['id']).group())
    print(pattern1.sub('', news.find().text))
    for a in news.select('a'):
        print(a.text.strip())
        print(urljoin(seed, a['href']))

100
정치
[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”
https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=081&aid=0003114437&date=20200810&type=1&rankingSeq=1&rankingSectionId=100
[속보] 김진애 "다주택자 왜 적대시하냐, 노영민 과했다"
https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=215&aid=0000892691&date=20200810&type=1&rankingSeq=2&rankingSectionId=100
문대통령 "집값 진정 양상…부동산감독기구 설치"(종합)
https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=001&aid=0011804920&date=20200810&type=1&rankingSeq=3&rankingSectionId=100
[속보]文대통령 "주택 문제, 당면한 최고의 민생 문제"
https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=277&aid=0004734195&date=20200810&type=0&rankingSeq=4&rankingSectionId=100
"티셔츠가 왜 이리 깨끗?" 심상정 수해복구 인증샷 구설
https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003552760&date=20200810&type=1&rankingSeq=5&rankingSectionId=100
수해현장 달려갔

In [11]:
# aid: article id

In [12]:
for news in dom.select('div[id^=ranking_]'):
    print(re.search('\d{3}', news['id']).group())
    print(pattern1.sub('', news.find().text))
    for a in news.select('li > a'):
        link = urljoin(seed, a['href'])
        print(a.text.strip())
        print(re.search('aid=(\d+)', link).group())

100
정치
[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”
aid=0003114437
[속보] 김진애 "다주택자 왜 적대시하냐, 노영민 과했다"
aid=0000892691
문대통령 "집값 진정 양상…부동산감독기구 설치"(종합)
aid=0011804920
[속보]文대통령 "주택 문제, 당면한 최고의 민생 문제"
aid=0004734195
"티셔츠가 왜 이리 깨끗?" 심상정 수해복구 인증샷 구설
aid=0003552760
수해현장 달려갔지만…`흙탕범벅` 태영호 `깔끔` 심상정
aid=0004632222
‘4주택자’ 김진애 “靑 참모들 집 매각은 노영민 개인 생각… 과했다”
aid=0003492251
영·호남 화합 상징 화개장터, 물난리 원인 놓고 여야 갈라섰다
aid=0003024870
'靑참모 줄사표' 언급 없이…文 "집값 진정됐다, 경제 선방했다"
aid=0003024878
폭우가 소환한 4대강…文 “홍수조절 효과 분석할 기회”(종합)
aid=0003114428
101
경제
정용진·백종원 '콜라보' 파워 무섭네…위기의 장어도 품절됐다
aid=0003024865
"집 안사면 되지 뭐"…더 뜨거워진 2030의 '수입차 플렉스'
aid=0004395676
'캄보디아 만삭 아내 살해사건' 남편에 보험금 100억원 지급되나
aid=0003780817
정용진·백종원 손 잡자…재고 900톤 바다장어 '품절 대란'
aid=0004734166
文이 5000만원 투자한 그 펀드, 1년 만에 2800만원 벌었다
aid=0003024787
품절…품절… 백종원·정용진 '바다장어'로 또 대박 냈다
aid=0004452947
실업급여 1조2천억 또 사상 최대…제조업은 외환위기 수준 실업
aid=0004632171
“우리가 호구냐”…8·4대책에 과천이어 마포주민 ‘뿔났다’
aid=0004710768
남부 폭우 가축에도 '대재앙'…타고난 수영선수 한우 '떼죽음'
aid=0011804967
16% 급등한 현대차, 코스피 2년 2개월만에 최고치
aid=0003552823
1

In [13]:
for news in dom.select('div[id^=ranking_]'):
    print(re.search('\d{3}', news['id']).group())
    print(pattern1.sub('', news.find().text))
    for a in news.select('li > a'):
        link = urljoin(seed, a['href'])
        print(a.text.strip())
        print(re.search('aid=(\d+)', link).group(1))

100
정치
[속보] 문 대통령 “집값 진정 양상…부동산 감독기구 설치 검토”
0003114437
[속보] 김진애 "다주택자 왜 적대시하냐, 노영민 과했다"
0000892691
문대통령 "집값 진정 양상…부동산감독기구 설치"(종합)
0011804920
[속보]文대통령 "주택 문제, 당면한 최고의 민생 문제"
0004734195
"티셔츠가 왜 이리 깨끗?" 심상정 수해복구 인증샷 구설
0003552760
수해현장 달려갔지만…`흙탕범벅` 태영호 `깔끔` 심상정
0004632222
‘4주택자’ 김진애 “靑 참모들 집 매각은 노영민 개인 생각… 과했다”
0003492251
영·호남 화합 상징 화개장터, 물난리 원인 놓고 여야 갈라섰다
0003024870
'靑참모 줄사표' 언급 없이…文 "집값 진정됐다, 경제 선방했다"
0003024878
폭우가 소환한 4대강…文 “홍수조절 효과 분석할 기회”(종합)
0003114428
101
경제
정용진·백종원 '콜라보' 파워 무섭네…위기의 장어도 품절됐다
0003024865
"집 안사면 되지 뭐"…더 뜨거워진 2030의 '수입차 플렉스'
0004395676
'캄보디아 만삭 아내 살해사건' 남편에 보험금 100억원 지급되나
0003780817
정용진·백종원 손 잡자…재고 900톤 바다장어 '품절 대란'
0004734166
文이 5000만원 투자한 그 펀드, 1년 만에 2800만원 벌었다
0003024787
품절…품절… 백종원·정용진 '바다장어'로 또 대박 냈다
0004452947
실업급여 1조2천억 또 사상 최대…제조업은 외환위기 수준 실업
0004632171
“우리가 호구냐”…8·4대책에 과천이어 마포주민 ‘뿔났다’
0004710768
남부 폭우 가축에도 '대재앙'…타고난 수영선수 한우 '떼죽음'
0011804967
16% 급등한 현대차, 코스피 2년 2개월만에 최고치
0003552823
102
사회
'보험금 95억' 아내 사망사고 낸 남편 금고2년…보험사기는 무죄(종합)
0011805029
제5호 태풍 '장미' 오후 2시 50분 

In [14]:
import os
os.getcwd()

'C:\\Users\\Shin\\Desktop\\kr-for-git\\5_data_analysis'

In [15]:
urls = ['https://news.naver.com']
visited = list()

#path = './naver/'
path = './'

while urls:
    seed = urls.pop(0) # Queue
    visited.append(seed)
    
    dom = BeautifulSoup(get(seed).text, 'html.parser')
    body = dom.select_one('#articleBodyContents')
    
    if body: # 뉴스
        cid = re.search('rankingSectionId=(\d+)', seed).group(1)
        aid = re.search('aid=(\d+)', seed).group(1)
        
        file = '{0}-{1}.txt'.format(cid, aid)
        with open(path+file, 'w', encoding='utf-8') as f:
            f.write(body.text)
            
    else: # 링크
        for a in dom.select('div[id^=ranking_] li > a'):
            link = urljoin(seed, a['href'])
            if link not in urls and link not in visited:
                urls.append(link)

In [16]:
urls = ['https://news.naver.com']
visited = list()

path = './'

while urls:
    seed = urls.pop(0) #Queue
    visited.append(seed)
    
    dom = BeautifulSoup(get(seed).text, 'html.parser')
    body = dom.select_one('#articleBody') #뉴스를 가져옴
    if body:
        cid = re.search('aid=(\d+)', seed).group(1)
        aid = re.search('rankingSectionId=(\d+)', seed).group(1)
        
        file = '{0}-{1}.txt'.format(cid,aid)
        with open(path+file, 'w', encoding = 'utf-8') as f:
            f.write(body.text)
        body.text
    else: #링크를 가져와야 함
        for a in news.select('div[id^=ranking_] li > a'):
            link = urljoin(seed, a['href'])
            if link not in urls and link not in visited:
                urls.append(link)

## 1-2. Indexer

In [17]:
import re
from string import punctuation
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))
pattern2 = re.compile(r'\b\w+@(?:[.]?\w+)+b')
pattern3 = re.compile(r'\bhttps?://\w+(?:[.]?\w+)+\b')
pattern4 = re.compile(r'[A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ ]')
pattern5 = re.compile(r'\b[A-Z][A-Za-z0-9]+\b')
pattern6 = re.compile(r'\s{2,}')

In [19]:
from os import listdir
from konlpy.tag import Okt
okt = Okt()

def fileids(path = './'):
    return [path+_ for _ in listdir(path) if re.search('[.]txt$', _)]

def cleaning(doc):
    return pattern6.sub(' ',
            pattern1.sub(' ',
            pattern5.sub(' ',
            pattern4.sub(' ',
            pattern2.sub(' ', doc))))).strip()

def tokenizer1(doc): # 어절
    return doc.split()

def tokenizer2(tokens, n=2): # 어절 ngram
    ngram = list()
    for i in range(len(tokens) - (n-1)):
        ngram.append(tokens[i:i+n])
    return ngram

def tokenizer3(doc, n=2): # 음절 ngram
    ngram = list()
    for i in range(len(doc) - (n-1)):
        ngram.append(doc[i:i+n])
    return ngram

def tokenizer4(doc): # 형태소
    return [_ for _ in okt.morphs(doc) if 1 < len(_) < 8]

def tokenizer5(doc): # 명사
    return [_ for _ in okt.nouns(doc) if 1 < len(_) < 8]

TypeError: startJVM() got an unexpected keyword argument 'convertStrings'

In [49]:
len(fileids())

120

In [50]:
with open(fileids()[0], 'r') as f:
    doc = f.read()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 19: illegal multibyte sequence

In [ ]:
doc = pattern2.sub(' ', doc)
doc = pattern3.sub(' ', doc)
doc = pattern4.sub(' ', doc)
doc = pattern5.sub(' ', doc)
doc = pattern1.sub(' ', doc)
pattern6.sub(' ', doc)

In [ ]:
with open(fileids[0], 'r') as f:
    doc = pattern6.sub(' ',
            pattern1.sub(' ',
            pattern5.sub(' ',
            pattern4.sub(' ',
            pattern2.sub(' ', f.read()))))).strip()

In [ ]:
doc

In [ ]:
with open(fileids()[0], 'r') as f:
    news = f.read()

In [ ]:
from collections import defaultdict
tokens = defaultdict(lambda:0)
for _ in tokenizer1(news):
    tokens[_] += 1
print(len(tokens), sun(tokens.values()))
for _ in tokenizer2(news):
    tokens[_] += 1
print(len(tokens), sun(tokens.values()))
for _ in tokenizer3(news):
    tokens[_] += 1
print(len(tokens), sun(tokens.values()))
for _ in tokenizer4(news):
    tokens[_] += 1
print(len(tokens), sun(tokens.values()))
for _ in tokenizer5(news):
    tokens[_] += 1
print(len(tokens), sun(tokens.values()))